In [ ]:
import torch
# from torch.autograd import Variable # Deprecated -> Torch에 지원
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset # DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

# 1. Wine 데이터 

### 데이터 준비

In [ ]:
wine = load_wine()
wine.keys()

In [ ]:
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df.head(3)

In [ ]:
df.shape, wine.target_names, wine.target

In [ ]:
 wine.target

In [ ]:
wine_data = wine.data[:130]
wine_target = wine.target[:130]
wine_data, wine_target

In [ ]:
# train_X, test_X, train_y, test_y = train_test_split(wine_data, wine_target, test_size=.2)
train_X, test_X, train_y, test_y = train_test_split(wine_data, wine_target, test_size=.2)

In [ ]:
print(train_X.shape)
print(test_X.shape)

### 텐서 생성

In [ ]:
# This is the Code
# tensor from numpy
train_X = torch.from_numpy(train_X).float() # double(float64)->float(float32)
train_y = torch.from_numpy(train_y).long() # long -> long (int64)

In [ ]:
test_X = torch.from_numpy(test_X).float()
test_y = torch.from_numpy(test_y).long() 

In [ ]:
print(train_X.size()) # ->104행 13열 (2차원배열)
print(train_y.size()) # ->104행 1열  (1차원배열)

In [ ]:
len(wine_data)

In [ ]:
# This is the Code
train = TensorDataset(train_X, train_y) # like Dataframe

In [ ]:
train

In [ ]:
train.tensors

In [ ]:
print("train_X", train.tensors[0].size())
print("train_y", train.tensors[1].size())

In [ ]:
# This is the Code
train_loader = DataLoader(train, batch_size=16, shuffle=True) 
train_loader

### 신경망 구성 - P.79

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)
    


In [ ]:
# model Instance 생성
model = Net()

### 모형 학습 - P.81

In [ ]:
# Cross Entropy Loss(Cost) Function
criterion = nn.CrossEntropyLoss()
# metric = nn.CrossEntropyLoss()

In [ ]:
# Stochastic Gradient Decent
optimizer = optim.SGD(model.parameters(), lr=0.01) # learning late

In [ ]:
for epoch in range(300):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        # train_x, train_y = Variable(train_x), Variable(train_y) # Deprecated
        # train_x.requires_grad_()
        
        # RuntimeError: only Tensors of floating point dtype can require gradients
        # train_y = train_y.float()
        # train_y.requires_grad_()
        # train_y = Variable(train_y)
        
        optimizer.zero_grad() # reset gradient
        
        output = model(train_x)
        
        loss = criterion(output, train_y)

        loss.backward()
        
        optimizer.step()
        
        # total_loss += loss.data[0]
        total_loss += loss.item()
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)
        print("="*40)

In [ ]:
type(test_X)

In [ ]:
# 출력이 0 혹은 1이 되게 함
result = torch.max(model(test_X), 1)[1]
# result = torch.max(model(test_X), 1).indices

#정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

# 2. MNIST 데이터

In [ ]:
import torch

# from torch.autograd import Variable  # Deprecated
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

import pandas as pd

# matplotlib 라이브러리 임포트
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
# mldata.org is no longer operational.
# mnist = datasets.fetch_mldata('MNIST original', data_home='./data/') # deprecated

In [ ]:
# https://www.openml.org
# https://www.openml.org/d/554
mnist = datasets.fetch_openml(name='mnist_784', data_home='./data/') # 사용하세요.

In [ ]:
mnist.keys()

In [ ]:
mnist.data.min(), mnist.data.max()

In [ ]:
# 값의 범위가 0~255
# 정규화 0~1사이로 만듬
mnist_data = mnist.data / 255

# column 784 --> 28*28
mnist_df = pd.DataFrame(mnist_data)
mnist_df.shape

In [ ]:
# p.85:
# 데이터가 없음.
plt.imshow(mnist_data[0].reshape(28, 28), cmap=cm.gray_r)

In [ ]:
mnist_label = mnist.target
mnist_label

In [ ]:
# 7만개
mnist_label.shape 

In [ ]:
train_size = 5000

test_size = 500

train_X, test_X, train_y, test_y = model_selection.train_test_split(
    mnist_data, mnist_label, train_size=train_size, test_size=test_size
)

### 텐서 생성 - p.86

In [ ]:
import numpy as np

train_X = torch.from_numpy(train_X).float()
# train_y = torch.from_numpy(train_y).long() Error
train_y = torch.from_numpy(train_y.astype(np.int)).long()

test_X = torch.from_numpy(test_X).float()
test_y = torch.from_numpy(test_y.astype(np.int)).long()

In [ ]:
print(train_X.shape)
print(train_y.shape)

In [ ]:
# feature + label
# DataLoader를 사용하기 위해 하는 작업
train = TensorDataset(train_X, train_y)

print(train[0])

# DataLoader: 미니배치를 가능하게 해줌.
train_loader = DataLoader(train, batch_size=100, shuffle=True)

## 신경망 구성

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.dropout(x, training=self.training)
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)
        


In [ ]:
model = Net()

## 모형 학습 - p.91

In [ ]:
# loss 함수
# Cross Entropy 손실함수(cost function)
criterion = nn.CrossEntropyLoss()

# 옵티마이저
# Stochastic Gradient Descent

optimizer = optim.SGD(model.parameters(), lr=0.01) # lr = learning rate

# 학습 시작
for epoch in range(1000):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        # grad 초기화
        optimizer.zero_grad()
        
        # Forward - 순전파
        output = model(train_x)
        
        loss = criterion(output, train_y)
        # backward - 역전파
        loss.backward()
        
        # 가중치 업데이트
        optimizer.step()
        
        # 오차 계산
        total_loss += loss.item()
        
    if (epoch +1) % 100 == 0:
        print(epoch+1, total_loss)
        
# 오래 걸려요..

In [ ]:
# classfication (예측)
# torch.exp(model(test_X).data)[1]

In [ ]:
plt.imshow(mnist_data[0].reshape(28, 28), cmap=cm.gray_r)

In [ ]:
torch.max(model(test_X).data, 1)[1]

In [ ]:
result = torch.max(model(test_X).data, 1)[1]


In [ ]:
result[:3]

In [ ]:
test_y[:3]

In [ ]:
# 정확도
accuracy = sum(test_y.data.numpy() == result.numpy())/ len(test_y.data.numpy())

accuracy

# 뉴스 기사 분류 - p.95

### 학습데이터 준비 p.20
- http://kristalinfo.com/TestCollections/#hkib

In [ ]:
# http://kristalinfo.com/TestCollections/#hkib
import os
import requests
import tarfile

gzip_url = 'http://kristalinfo.com/download/hkib-20000-40075.tar.gz'

filename = './data/hkib/hkib-20000-40075.tar.gz'
base_dir = os.path.dirname(filename)

if not os.path.exists(os.path.dirname(filename)):
    os.makedirs(os.path.dirname(filename))

# 크롤링
with requests.get(gzip_url, stream=True) as resp:
    with open(filename, 'wb') as f:
        while True:
            chunk = resp.raw.read(1024, decode_content=True)
            if not chunk:
                break
            f.write(chunk)



In [ ]:
# 압축 풀기
with tarfile.open(filename, "r:gz") as tar:
    tar.extractall(base_dir)

## 데이터 import - p.96

In [ ]:
import os 

# 정규표현식
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# pandas
import pandas as pd

# numpy
import numpy as np

In [3]:
!pip install JPype1-0.6.2-cp36-cp36m-win_amd64.whl

Processing c:\users\student\desktop\multicampus_lecture\pytorch\jpype1-0.6.2-cp36-cp36m-win_amd64.whl


ERROR: Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: 'C:\\Users\\student\\Desktop\\multicampus_lecture\\pytorch\\JPype1-0.6.2-cp36-cp36m-win_amd64.whl'



In [ ]:
!pip install JPype1
# !pip install konlpy

In [ ]:
# konlpy
# 형태소 분석기 
try:
    from konlpy.tag import Hannanum, Kkma
except ModuleNotFoundError as e:
    import sys
    !{sys.executable} -m pip install konlpy
finally:
    from konlpy.tag import Hannanum, Kkma
    


In [ ]:
base_dir

In [ ]:
import glob
target_dir = os.path.join(base_dir, 'HKIB-20000')
cat_dirs = ['health', 'economy', 'science', 'education', 'culture', 'society', 'industry', 'leisure', 'politics']
cat_prefixes = ['건강', '경제', '과학', '교육', '문화', '사회', '산업', '여가', '정치']

files = os.listdir(target_dir)
print(files)

for file in files:
    if not file.endswith('.txt'):
        continue

    # 기사 단위 분할 -> 리스트
    with open(target_dir + '/' + file) as curfile:
        doc_cnt = 0
        docs = []
        curr_doc = None
        
        for curr_line in curfile:
            # 매 라인 읽기
            if curr_line.startswith('@DOCUMENT'):
                # 시작점 찾기
                if curr_doc is not None:
                    # 읽고 있는 문서가 있으면 붙이기
                    docs.append(curr_doc)
                    
                curr_doc = curr_line
                doc_cnt = doc_cnt +1
                continue
            curr_doc = curr_doc + curr_line

        # 마지막 것 넣어줘야함 (책 오류)
        docs.append(curr_doc)
        print(len(docs), docs[-1]==docs[-2])

    # 기사 분류
    for doc in docs:
        doc_lines = doc.split('\n')
        doc_no = doc_lines[1][9:] # doc_lines[1].split(':')[1].strip()
        
        # 주제 추출
        doc_cat03 = ''
        for line in doc_lines[:10]:
            if line.startswith("#CAT'03"):
                doc_cat03 = line[10: ]
                break
        
        # 주제별로 디렉토리 정리
        for cat_prefix in cat_prefixes:
            if doc_cat03.startswith(cat_prefix):
                # prefix로 시작하는지 확인
                dir_index = cat_prefixes.index(cat_prefix) # index 맞춰서 저장 --> 폴더명은 영문
                break
        
        # 문서 정보 제거, 기사 본문만 남기기
        filtered_lines = []
        for line in doc_lines:
            if not (line.startswith('#') or line.startswith('@')):
                # 메타태그 제외 append
                filtered_lines.append(line)
        
        filepath = os.path.join(target_dir, cat_dirs[dir_index])
        filename = os.path.join(filepath, 'hkib-' + doc_no + '.txt')
        
        
        if not os.path.exists(filepath):
            os.makedirs(filepath)

        with open(filename, 'w') as f:
            f.write('\n'.join(filtered_lines))
        
            


In [ ]:
# 예제 6.16~6.17
# 예제 6.17로 진행. (6.16은 의미X)
dirs = ['education', 'health']

# 기사에 출현하는 단어와 레이블 저장 리스트
# features
x_ls = []

# target_values
y_ls = []

# 필요없음
# tmp1 = [] # 임시 토큰 저장소
# tmp2 = '' # 임시 문자열 저장소
tmp = ''

# tokenizer = Hannnum()
tokenizer = Kkma()

#각 폴더의 파일을 하나씩 읽음
for i, d in enumerate(dirs):
    
    # 해당 디렉토리에 파일들 ['education', 'health']
    base_dir = target_dir + '/' + d
    files = os.listdir(base_dir)
    
    for fid, file in enumerate(files):
        with open(base_dir + '/' + file, 'r', encoding='utf-8') as f:
            raw = f.read()
            
            # reg_raw = re.sub(r'[0-9a-zA-Z]', '', raw) # 숫자 및 영어 제거 (? - 안해도 괜찮아 보임)
            reg_raw = re.sub(r'[-\'@#:/◆▲0-9a-zA-Z<>!-"*\(\)]', '', raw) # 윈도우는 [ㅁ+한자키], 맥은 [cmd+ctrl+space
            
            reg_raw = re.sub(r'[ ]+', ' ', reg_raw)
            reg_raw = reg_raw.replace('\n', ' ') # 줄바꿈 없음
        
        # 형태소 분석 -> 명사만 추출하여 리스트 생성
        tokens = tokenizer.nouns(reg_raw)
        
        tmp = ' '.join(tokens)
        
        # 필요 없음
        # for token in tokens: # 토큰 저장
        # tmp1.append(token)
        
        x_ls.append(tmp)
        y_ls.append(i) # 기사 주제 레이블 (label index)
        if fid % 50 == 0:
            print("{}-[{}/{}] 진행중...".format(i+1, fid+1, len(files)+1))
        
            

In [ ]:
len(x_ls), len(y_ls)

In [ ]:
pd.DataFrame(x_ls).head()

In [ ]:
print(x_ls[0])

In [ ]:
print(y_ls)

In [ ]:
## numpy 배열
x_array = np.array(x_ls)
y_array = np.array(y_ls)

## 단어 출현 횟수 세기
# sklearn 변환기 # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
cntvec = CountVectorizer() 
x_cntvecs = cntvec.fit_transform(x_array)
x_cntarray = x_cntvecs.toarray() # matrix.toarray()

# 데이터 프레임으로 변환
df = pd.DataFrame(x_cntarray)
df.head()

In [ ]:
### 단어와 단어의 인덱스 표시

## CountVectorizer().vocabulary_: 단어와 인덱스 매핑
## dict 형식: { '<단어>': <index>, ...}
for k, v in sorted(cntvec.vocabulary_.items(), key=lambda x: x[1]):
    # 사전식 정렬
    print(k, v)


In [ ]:
# TfidfVectorizer =>
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidf_vec = TfidfVectorizer(use_idf=True)
x_tfidf_vecs = tfidf_vec.fit_transform(x_array)
x_tfidf_array = x_tfidf_vecs.toarray()

pd.DataFrame(x_tfidf_array).head()

#### 데이터 split

In [ ]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(x_tfidf_array, y_array, test_size=.2, random_state=42)
train_X.shape, test_X.shape

### 텐서 생성 - p.106

In [ ]:
# Pytorch 임포트
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

In [ ]:
train_X = torch.from_numpy(train_X,).float()
# train_y = torch.from_numpy(train_y).long()
train_y = torch.from_numpy(train_y).long() # only 

test_X = torch.from_numpy(test_X,).float()
test_y = torch.from_numpy(test_y).long() # only 

train_X.size(), train_y.size()

In [ ]:
train = TensorDataset(train_X, train_y)

print(train[0])

train_loader = DataLoader(train, batch_size=100, shuffle=True)

### 신경망 구성

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(33572, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=0)
    

In [ ]:
model = Net()
# model.cuda() 우린 GPU가 없어요.

### 학습하기 - p.111

In [ ]:
criterion = nn.CrossEntropyLoss()

#optimizer: (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.005)

# learning
for epoch in range(100):
    total_loss =0
    
    # 미니배치
    for train_x, train_y in train_loader:
        # train_x.requires_grad_()
        # train_y.requires_grad_()

        # slope init
        optimizer.zero_grad()
        
        # forward
        output = model(train_x)
        
        # 오차
        loss = criterion(output, train_y)
        
        # backward(역전파)  계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        
        total_loss += loss.item()
        
    print(epoch+1, total_loss)
    

In [ ]:
test_x, test_y = Variable(test_X), Variable(test_y)
result = torch.max(model(test_x).data, 1)[1] # 모델에서 가장 큰 값으로

# test_y.cpu() 우린 GPU를 안 썼다.
# accuracy = sum(test_y.data.cpu().numpy() == result.cpu().numpy()) / len(test_y.cpu().data.numpy())
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

# 시계열 데이터에서 이상탐지하기 - p.114

##### 2011. 1월 1일부터 2019년 06월 25일까지 데이터
#### https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36

### 데이터 준비

In [ ]:
import os 
from torch import nn
import torch.optim as optim


# 정규표현식
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# pandas
import pandas as pd

# numpy
import numpy as np

In [ ]:
target_dir = './data/weather'
filename = '20110101_20190515.csv'
filepath = os.path.join(target_dir, filename)

In [ ]:
import torch
# from torch.autograd import Variable # Deprecated
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import pandas as pd

import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
dat = pd.read_csv(filepath, encoding='cp949')
dat

In [ ]:
# 8.5년
temp = dat['평균기온(°C)']

temp.plot()
plt.show()

In [ ]:
# 8.5년
train_x = temp[:int(365*6.5)]
test_x = temp[int(365*6.5):]

train_x = np.array(train_x)
test_x = np.array(test_x)

In [ ]:
len(train_x), len(test_x)

In [ ]:
ATTR_SIZE = 180 
train_x[0:0+ATTR_SIZE][:3]

In [ ]:
train_x[1:1+ATTR_SIZE][:3]

In [ ]:
ATTR_SIZE = 180 # 윈도우 폭

tmp=[]
train_X = []

for i in range(0, len(train_x) - ATTR_SIZE):
    tmp.append(train_x[i:i+ATTR_SIZE]) ## 윈도우 폭만큼 데이터 저장.

train_X = np.array(tmp)
df = pd.DataFrame(train_X)
df.head()

### 신경망 구성

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(180, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, 180)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
model = Net()

### 모형 학습

In [ ]:
criterion = nn.MSELoss()

# optim.SGD
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(7000):
    total_loss = 0
    d = []
    
    for i in range(100):
        # 훈련 데이터에 랜덤으로 인덱스 부여
        index = np.random.randint(0, len(train_X))
        
        #미니배치 분할
        d.append(train_X[index])
    d = np.array(d, dtype='float32')
    
    d = torch.from_numpy(d)
    
    # zero grad
    optimizer.zero_grad()
    # forward
    output=model(d)
    
    # loss
    loss = criterion(output, d)
    #backward
    loss.backward()
    
    optimizer.step()
    total_loss += loss.item()
    
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

In [ ]:
plt.plot(d.data[0].numpy(), label='original')
plt.plot(output.data[0].numpy(), label='output')
plt.legend(loc='upper right')
plt.show()

### 이상 점수 계산 - p.126

In [ ]:
len(test_x)

In [ ]:
tmp = []
test_X = []
last_i = 0

# 테스트 데이터 분할 180
print(len(test_x))
for i in range(0, len(test_x), 180):
    if len(test_x) - i < 180:
        # 개수 안맞으면 break
        break
    tmp.append(test_x[i:i+180])
    last_i = i+180

test_X = np.array(tmp, dtype='float32')
pd.DataFrame(test_X)
# tmp.append(test_x[])

In [ ]:
d = torch.from_numpy(test_X)
output = model(d)

# ploting
plt.plot(test_X.flatten(), label='original')
plt.plot(output.data.numpy().flatten(), label='prediction')

plt.legend(loc='upper right')
plt.show()

In [ ]:
# 이상 점수 계산
test = test_X.flatten()
pred = output.data.numpy().flatten()

total_score = []

for i in range(0, last_i):
    dist = test[i] - pred[i]
    # x to the power y,
    score = pow(dist, 2) # dist **2 
    total_score.append(score)
    
# 이상 점수 정규화 [0, 1]
total_score = np.array(total_score)
max_score = np.max(total_score)
total_score = total_score / max_score

total_score

In [ ]:
# 임계값을 정해둬야 함.
plt.plot(total_score)
plt.show()